In [ ]:
# Установка библиотеки aiogram версии 2.23.1
!pip install --force-reinstall -v "aiogram==2.23.1"

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.4/164.4 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.3/124.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.6/301.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 1.2 MB/s eta 0:00:00
  Attempting uninstall: py

In [ ]:
# Установка необходимых библиотек
import nest_asyncio
nest_asyncio.apply()

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch

import logging
import aiogram
from aiogram import Bot, Dispatcher, types
from aiogram.utils import executor

from aiogram.types import Message
from aiogram.utils.markdown import hbold

from aiogram.types import ReplyKeyboardMarkup, KeyboardButton, InlineKeyboardMarkup, InlineKeyboardButton

import zipfile
import os

from google.colab import files
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# Загрузка первой модели и токенизатора
model_bart = AutoModelForSeq2SeqLM.from_pretrained("/content/gdrive/My Drive/my_model")
tokenizer_bart = AutoTokenizer.from_pretrained("/content/gdrive/My Drive/my_tokenizer")

In [ ]:
# Загрузка файла второй модели
uploaded_model = files.upload()
# Получаем путь к загруженному файлу
model_zip_path = next(iter(uploaded_model))

Saving my_model-20240605T193840Z-001.zip to my_model-20240605T193840Z-001.zip


In [ ]:
# Загрузка файла токенизатора
uploaded_tokenizer = files.upload()
# Получаем путь к загруженному файлу
tokenizer_zip_path = next(iter(uploaded_tokenizer))

Saving my_tokenizer-20240605T193828Z-001.zip to my_tokenizer-20240605T193828Z-001.zip


In [ ]:
# Распаковка zip-файла с моделью
with zipfile.ZipFile(model_zip_path, 'r') as model_zip:
    model_zip.extractall("/content/model/")

# Распаковка zip-файла с токенизатором
with zipfile.ZipFile(tokenizer_zip_path, 'r') as tokenizer_zip:
    tokenizer_zip.extractall("/content/tokenizer/")

In [ ]:
# Путь к модели
model_path = "/content/model/my_model"
# Загрузка модели
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

# Путь к токенизатору
tokenizer_path = "/content/tokenizer/my_tokenizer"
# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

In [ ]:
# Включаем логирование
logging.basicConfig(level=logging.INFO)

# Инициализация бота
TOKEN = 'ТОКЕН'
bot = Bot(token=TOKEN)
dp = Dispatcher(bot)

In [ ]:
# Словарь для хранения диалога пользователя
dialogue_history = {}

# Функция для обрезки истории диалога
def trim_history(history, max_length=4096):
    current_length = sum(len(message) for message in history.values())
    while history and current_length > max_length:
        removed_message = history.popitem(last=False)
        current_length -= len(removed_message[1])
    return history

In [ ]:
# Обработчик команды /start
@dp.message_handler(commands=['start'])
async def start_command(message: types.Message):
    # Создаем клавиатуру с кнопками для выбора языка
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True)
    btn1 = types.KeyboardButton("🇷🇺 Русский")
    btn2 = types.KeyboardButton("🇬🇧 English")
    markup.add(btn1, btn2)
    # Отправляем сообщение с клавиатурой
    await message.answer("Привет!👋 Я умею кратко пересказывать длинные диалоги.\n\n Выберите язык", reply_markup=markup)

@dp.message_handler(lambda message: message.text in ["🇷🇺 Русский", "🇬🇧 English"])
async def language_selected(message: types.Message):
    # Получаем выбранный язык
    selected_language = message.text.split(" ")[1]
    # Выполняем действия в зависимости от выбранного языка
    if selected_language == "Русский":
        # Обработка выбора русского языка
        await message.answer("Язык изменен на 🇷🇺")
    elif selected_language == "English":
        # Обработка выбора английского языка
        await message.answer("Language changed to 🇬🇧")

In [ ]:
# Обработчик команды /help
@dp.message_handler(commands=['help'])
async def help_command(message: types.Message):
    # Выводим список доступных команд
    help_text = "Доступные команды:\n\n"
    help_text += "/model - выбор модели для суммаризации \n"
    help_text += "/checkmodel - посмотреть название загруженной модели\n"
    help_text += "/clear - очистка истории\n"
    help_text += "/help - вывести список доступных команд\n"
    await message.answer(help_text)

In [ ]:
# Переменная для хранения текущей модели
current_model = model_bart
current_tokenizer = tokenizer_bart

# Обработчик команды /model
@dp.message_handler(commands=['model'])
async def model_command(message: types.Message):
    # Создаем клавиатуру с выбором моделей
    markup = types.InlineKeyboardMarkup()
    btn1 = types.InlineKeyboardButton("bart-base", callback_data='model_bart')
    btn2 = types.InlineKeyboardButton("flan-t5-base", callback_data='model')
    markup.add(btn1, btn2)
    await message.answer("Выберите модель для суммаризации:", reply_markup=markup)

# Обработчик выбора модели через инлайн-кнопки
@dp.callback_query_handler(lambda c: c.data in ['model_bart', 'model'])
async def process_model_selection(callback_query: types.CallbackQuery):
    global current_model, current_tokenizer
    # Обрабатываем выбор модели
    if callback_query.data == 'model_bart':
        # Загружаем модель 1
        current_model = model_bart
        current_tokenizer = tokenizer_bart
        await callback_query.answer("Модель BART выбрана")
    elif callback_query.data == 'model':
        # Загружаем модель 2
        current_model = model
        current_tokenizer = tokenizer
        await callback_query.answer("Модель T5 выбрана")

# Обработчик команды /checkmodel, который выводит текущую модель
@dp.message_handler(commands=['checkmodel'])
async def checkmodel_command(message: types.Message):
    # Получаем название текущей модели
    if current_model == model_bart:
        current_model_name = "bart-base"
    else:
        current_model_name = "flan-t5-base"
    await message.answer(f"Текущая модель: {current_model_name}")

In [ ]:
# Обработчик команды /clear
@dp.message_handler(commands=['clear'])
async def process_clear_command(message: types.Message):
    user_id = message.from_user.id
    dialogue_history[user_id] = []
    await message.reply("История очищена.")

In [ ]:
# Обработчик всех остальных сообщений, который выполняет суммаризацию
@dp.message_handler()
async def summarize_handler(message: types.Message) -> None:
    """
    Handler that summarizes the received message using a pre-trained model
    """
    try:
        user_input = message.text
        # Проверяем, что текст сообщения не содержит непонятных знаков
        if not all(ord(char) < 128 for char in user_input):
            await message.answer("Извините, я Вас не понимаю.")
            return
        # Добавляем сообщение в историю диалога
        dialogue_history[len(dialogue_history) + 1] = user_input
        # Генерируем краткий пересказ
        input_ids = current_tokenizer.encode(user_input, return_tensors='pt')
        output_ids = current_model.generate(input_ids, max_length=200, num_return_sequences=1, early_stopping=True)
        summary = current_tokenizer.decode(output_ids[0], skip_special_tokens=True)
        # Проверяем, что сводка не пустая перед отправкой
        if summary:
            # Отправляем пересказ пользователю
            await message.answer(f"Summary: {summary}")
        else:
            raise ValueError("Сводная информация не была получена.")
    except Exception as e:
        logging.error(f"Ошибка в summarize_handler: {e}")
        await message.answer("Извините, произошла ошибка.")

# Основная функция для запуска бота
def main() -> None:
    executor.start_polling(dp, skip_updates=True)

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
